# <center><b>Regression template</b></center>

## <b>Step 0:</b> Context
*Explain the problem you're trying to solve, the nature of the dataset, the goal of the model, and the impact of the solution.*

## <b>Step 1:</b> Imports and configurations

### <b>Libraries</b>
*Import all necessary Python libraries (e.g., numpy, pandas, scikit-learn, tensorflow/pytorch for deep learning).*

In [ ]:
#- Workflow
from sklearn.pipeline import Pipeline
import mlflow
import mlflow.sklearn

#- Data manipulation
import pandas as pd
import numpy as np

#- Data visualisation
import matplotlib.pyplot as plt
import seaborn as sns

#- Preprocessing for model fitting
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split

#- ML models (for regression)
import optuna
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor

#- Model assessment for regression
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
)
from sklearn.model_selection import cross_val_score, KFold
from scipy import stats


### <b>Dataset</b>
*If you have several data sets, you need to duplicate the cells below. If your dataset is in a different format, create a new cell and write the code to import it. Don't forget Chat GPT and Google are your friends. ;-)*

*If your data is in a CSV file use the cell below to import it.*

In [ ]:
data_path = ".csv"
data = pd.read_csv(data_path)
raw_data = data.copy()
data.head(3)

*If your data is in an excel file use the cell below to import it instead.*

In [ ]:
# data_path = ".xlsx"
# data = pd.read_excel(data_path)
# data.head(3)

### <b>Configurations</b>
*Set any global configurations such as random seed for reproducibility, backend settings for computation libraries, and any project-specific parameters.*

In [ ]:
SEED = 42

#- Show all the column
pd.set_option('display.max_columns', None)

#- Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("test_mlflow_setup")      # This is just to test if the mlflow server is working
try:
    mlflow.log_param("test_param", "test_value")
    print("MLflow is configured correctly!")
except Exception as e:
    print(f"MLflow configuration failed: {e}")


## <b>Step 2:</b> Data inspection and cleanings

### <b>Inspection</b>
*The goal of data inspection is to understand the dataset's characteristics, quality, and any potential issues that need addressing before analysis.*

In [ ]:
def basic_inspection(df, visualize_nulls=True, detailed_summary=True, check_duplicates=True):
    """
    Perform a basic inspection of a pandas DataFrame including size, data types,
    null values, a heatmap of missing values, duplicate rows, and a summary table with optional
    detailed analysis.

    Parameters:
    - df: pandas DataFrame.
    - visualize_nulls: Bool, if True, visualizes null values using a heatmap.
    - detailed_summary: Bool, if True, includes detailed analysis in the summary table.
    - check_duplicates: Bool, if True, checks and reports the number of duplicate rows.

    Returns:
    - A pandas DataFrame containing a summary of the inspection.
    """
    print(f'Number of rows: {df.shape[0]}\nNumber of columns: {df.shape[1]}\n')
    print('Data types:\n', df.dtypes)
    
    null_counts = df.isnull().sum()
    print('\nColumns with null values:\n', null_counts[null_counts > 0])

    if check_duplicates:
        duplicate_rows = df.duplicated().sum()
        print(f'\nNumber of duplicate rows: {duplicate_rows}\n')

    if visualize_nulls:
        plt.figure(figsize=(15, 7))
        sns.heatmap(df.isna(), cbar=True, cmap='viridis')
        plt.title('Heatmap of Missing Values')
        plt.show()

    summary_table = pd.DataFrame({
        "Unique_values": df.nunique(),
        "Data_type": df.dtypes,
        "Null_count": df.isnull().sum(),
        "Null_percentage": (df.isnull().sum() / df.shape[0] * 100).round(2)
    })

    if detailed_summary:
        if 'object' in df.dtypes.values:
            summary_table['Most_common'] = df.apply(lambda x: x.value_counts().idxmax() if x.dtypes == 'object' else 'unknown')
        for col in df.select_dtypes(include=['int64', 'float64']).columns:
            summary_table.loc[col, 'Mean'] = df[col].mean()
            summary_table.loc[col, 'Std'] = df[col].std()
            summary_table.loc[col, 'Min'] = df[col].min()
            summary_table.loc[col, 'Max'] = df[col].max()

    return summary_table

*For the following cell, modify the parameters according to the particularities of your database. For example, if you have intentional duplicates, it is not necessary to check them.*

In [ ]:
basic_inspection(data, visualize_nulls=True, detailed_summary=True, check_duplicates=True)

In [ ]:
def identify_outliers(df, plot=True):
    """
    Identify outliers in a pandas DataFrame using Z-score (values more than 3 standard deviations from 
    the mean) and Interquartile Range (IQR) methods (values below Q1 - 1.92*IQR or above Q3 + 1.92*IQR).
    Outliers identified by both methods are combined to provide a comprehensive overview of outliers 
    in each numeric feature.
    
    Parameters:
    - df (pd.DataFrame): A pandas DataFrame containing the data to be analyzed. Only numeric
      columns will be considered for outlier detection.
    
    Returns:
    - dict: A dictionary where each key is the name of a numeric feature in the DataFrame. Each value
      is another dictionary containing two keys: 'num_outliers', which is the number of unique outliers
      identified in the feature, and 'outliers_index', an array of indices of these outliers.
    """
    outlier_summary = {}
    
    for column in df.select_dtypes(include=np.number).columns:  # Focus on numeric columns
        # Calculate Z-scores
        z_scores = np.abs(stats.zscore(df[column].dropna()))
        z_outliers = np.where(z_scores > 3)[0]
        
        # Calculate IQR
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        iqr_outliers = df[(df[column] < (Q1 - 1.96 * IQR)) | (df[column] > (Q3 + 1.96 * IQR))].index
        
        # Combine unique outliers from both methods
        combined_outliers = np.union1d(z_outliers, iqr_outliers)
        
        # Optionally plot
        if plot:
            print(column)
            plt.figure(figsize=(8, 4))
            sns.boxplot(x=df[column])
            plt.title(f'Boxplot of {column} (Outliers highlighted)')
            plt.show()

        # Summary
        outlier_summary[column] = {
            'num_outliers': len(combined_outliers),
            'outliers_index': combined_outliers
        }
    
    return outlier_summary

*This function does not modify the original DataFrame. I recommend you to inspect the identified outliers and decide on appropriate handling methods such as removal, replacement, or keeping them as is, depending on the analysis requirements and domain knowledge.*

In [ ]:
outlier_info = identify_outliers(data)
print(outlier_info)

### <b> Cleaning</b>
*Following the inspection, data cleaning aims to rectify issues identified, improving the dataset's quality and making it suitable for further analysis and modeling.*

In [ ]:
data.head(3)

In [ ]:
#- Feature Selection

# data.drop(column=[], inplace=True)

In [ ]:
#- Remove duplicates

# data.drop_duplicates(inplace=True)

In [ ]:
def data_cleaning(data):
    #- Handle missing values
    data['numerical_column'].fillna(data['numerical_column'].mean(), inplace=True)
    data['categorical_column'].fillna(data['categorical_column'].mode()[0], inplace=True)
    
    #- Correct data type
    data['numeric_column_as_string'] = pd.to_numeric(data['numeric_column_as_string'], errors='coerce')
    data['date_column'] = pd.to_datetime(data['date_column'])

    #- Binning
    #- Fixed width binning
    data['value_bin'] = pd.cut(data['column_to_bin'], bins=3, labels=["Low", "Medium", "High"])

    #- Qauntile based binning
    data['value_quantile_bin'] = pd.qcut(data['column_to_bin'], q=3, labels=["Low", "Medium", "High"])

    #- Custom binning
    bins = []
    data['value_custom_bin'] = pd.cut(data['column_to_bin'], bins=bins, labels=["Low", "Medium", "High"], right=False)

    #- Turn categorical variables into numerical variables

    #- Label encoding
    # Define a mapping of categories to numerical values
    checking_mapping = {'None': 0, 'little': 1, 'moderate': 2, 'rich': 3}
    # Map the categories to their numerical equivalents
    data['col_name'] = data['col_name_numb'].map(checking_mapping)

    #- One-hot encoding of categorical variables
    categorical_cols = []       # Add the columns of interest
    data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
    return data

In [ ]:
data = data_cleaning(data)

<i>When deciding between one-hot encoding and simple encoding (label encoding) for categorical data, consider the nature of your categories:

- One-Hot Encoding: Use this when your categorical variable does not have a meaningful order or hierarchy. One-hot encoding creates a new binary column for each category, which is ideal for nominal data (e.g., color, city names). This approach prevents the model from assuming a natural ordering between categories, which is helpful to avoid misleading interpretations.

- Label Encoding: Use this when your categorical variable has a meaningful order or ranking (ordinal data). Label encoding assigns a unique integer to each category, preserving the order. However, be cautious, as this can introduce unintended ordinal relationships that may not exist in the data.</i>

Also don't apply the cleaning of the target feature in the data_cleaning function as we will apply this function to the prediction dataset (which won't have the target column) too. Do the transformation for the target column in the cell below

In [ ]:
#- Handle missing values
data['numerical_column'].fillna(data['numerical_column'].mean(), inplace=True)

#- Correct data type
data['numeric_column_as_string'] = pd.to_numeric(data['numeric_column_as_string'], errors='coerce')

In [ ]:
#- Outlier Treatment

# for col, info in outlier_info.items():
#     data = data.drop(index=info['outliers_index'], inplace=False)


In [ ]:
data.head(3)

## <b>Step 3:</b> Exploratory Data Analysis (problem navigation)

### <b> Univariate analysis</b>

In [ ]:
def quant_univariate_analysis(df, col_name, visualize=True, **kwargs):
    """
    Perform univariate analysis on a numerical column with visualization
    options using a Matplotlib native color palette focusing on red, yellow, and black.

    Parameters:
    df (DataFrame): Input DataFrame containing the data.
    col_name (str): Name of the numerical column to analyze.
    visualize (bool): Whether to visualize the analysis (default=True).
    **kwargs: Additional keyword arguments to customize the visualisation colors.

    Returns:
    pandas.Series: Descriptive statistics of the column.
    """
    if col_name not in df.columns:
        raise ValueError(f"Column '{col_name}' not found in the DataFrame.")

    # 1. Descriptive Statistics
    descriptive_stats = df[col_name].describe()
    print("\n"*2, col_name, "\n"*2, descriptive_stats)

    if visualize:
        # Custom color palette
        custom_colors = ['red', 'yellow', 'black']  # Red, Yellow, Black
        color = kwargs.get('hist_color', custom_colors[0])  # Use red as default
        
        # 2. Histogram with KDE
        plt.figure(figsize=kwargs.get('figsize', (12, 6)))
        sns.histplot(df[col_name], kde=True, bins=kwargs.get('bins', 30),
                     color=color,
                     kde_kws={'bw_adjust': kwargs.get('bw_adjust', 1)},
                     line_kws={'color': custom_colors[2], 'lw': 2})  # Use black for KDE line
        plt.title(f'Histogram and KDE of {col_name}')
        plt.xlabel(col_name)
        plt.ylabel('')
        plt.grid(True, linestyle='--', linewidth=0.5, color=custom_colors[1])  # Use yellow for grid lines
        plt.show()

        # 3. Boxplot
        plt.figure(figsize=kwargs.get('figsize', (12, 6)))
        sns.boxplot(x=df[col_name], color=kwargs.get('boxplot_color', custom_colors[1]))  # Use yellow for boxplot
        plt.title(f'Boxplot of {col_name}')
        plt.xlabel(col_name)
        plt.grid(True, linestyle='--', linewidth=0.5, color=custom_colors[2])  # Use black for grid lines
        plt.show()

    return descriptive_stats

In [ ]:
for quant_col in []:        # Add in the brackets the name of the quantitative variables in your dataset that you want to visualize.
    quant_univariate_analysis(data, quant_col)

In [ ]:
def qual_univariate_analysis(df, col_name, palette="viridis", show_grid=True, figsize=(12, 6)):
    """
    Performs and visualizes a univariate analysis for a qualitative (categorical) variable,
    highlighting and annotating the most and least common categories.

    Parameters:
    - df (DataFrame): The pandas DataFrame containing the data.
    - col_name (str): The name of the column to analyze.
    - palette (str, optional): Color palette for the plots. Defaults to 'viridis'.
    - show_grid (bool, optional): Whether to show the grid in the bar plot. Defaults to True.
    - figsize (tuple, optional): Figure size for the plots. Defaults to (12, 6).
    """
    
    # Frequency Table
    freq_table = df[col_name].value_counts()
    # Percentage Table
    percent_table = df[col_name].value_counts(normalize=True) * 100
    combined_table = pd.DataFrame({'Frequency': freq_table, 'Percentage': percent_table})
    print("\n"*2, col_name, "\n"*2, combined_table)

    # Identify most and least common categories
    most_common = freq_table.idxmax()
    least_common = freq_table.idxmin()

    # Bar Plot with Highlighting
    plt.figure(figsize=figsize)
    barplot = sns.countplot(y=df[col_name], order=freq_table.index, palette=palette)
    
    # Highlighting
    for patch in barplot.patches:
        if patch.get_y() == freq_table.index.get_loc(most_common):
            patch.set_facecolor('green')  # Highlight most common category in green
        elif patch.get_y() == freq_table.index.get_loc(least_common):
            patch.set_facecolor('red')  # Highlight least common category in red
    
    # Annotations
    plt.title(f'Bar Plot of {col_name}')
    plt.xlabel('Count')
    plt.ylabel(col_name)
    if show_grid:
        plt.grid(axis='x', linestyle='--', linewidth=0.5)
    
    # Adding annotations for the most and least common categories
    plt.text(freq_table.max(), freq_table.index.get_loc(most_common), 'Most common', fontsize=12, va='center')
    plt.text(freq_table.min(), freq_table.index.get_loc(least_common), 'Least common', fontsize=12, va='center')
    
    plt.show()

    return combined_table

In [ ]:
for qual_col in []:         # Add in the brackets the name of the qualitative variables in your dataset that you want to visualize.
    qual_univariate_analysis(data, qual_col)

### <b> Bivariate analysis</b>

In [ ]:
#- Calculate the correlation matrix
correlation_cols = []       # Add in the brackets the name of the the columns you want to visualize for correlation. Make sure these columns are numeric. You can numerize the qualitative columns back in the preprocessing step.
correlation = data[correlation_cols].corr(method='pearson')

#- Visualize the correlation matrix
fig, ax = plt.subplots()

ax.figure.set_size_inches(10, 10)
mask = np.triu(np.ones_like(correlation, dtype=bool))
cmap = sns.diverging_palette(220, 10, as_cmap=True)
sns.heatmap(correlation, cmap=cmap, mask=mask, square=True, linewidths=.5, 
            annot=True, annot_kws={'size':14})

plt.show()

In [ ]:
strong_corr = correlation[(correlation > 0.7) | (correlation < -0.7)]
print("Strong correlations:\n", strong_corr)

## <b>Step 4:</b> Feature Engineering
*The Feature engineering phase involves transforming raw data into meaningful features that effectively represent the underlying problem.*  
*<b>Feature Creation:</b>* Develop new features from the existing data to better capture the underlying patterns.  
*<b>Feature Transformation:</b>* Apply transformations (e.g., scaling, encoding) to make the data suitable for modeling.  
*<b>Feature Selection:</b>* Use statistical tests and selection algorithms to reduce dimensionality and focus on relevant features.

In [ ]:
def do_feature_eng(df):
    return df

In [ ]:
data = do_feature_eng(data)

## <b>Step 5:</b> Data preparation

### <b>Define Features and Target</b>

In [ ]:
#- Features:
features_cols = []
X = data[features_cols]

#- Target:
y = data[target_name]       # Replace target_name with the name of your target variable in your dataset.

### <b>Train validation and Test data</b>

In [ ]:
pre_X_train, X_test, pre_y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=SEED, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(pre_X_train, pre_y_train, test_size=0.25, random_state=42, stratify=pre_y_train)

In [ ]:
print("Size of the X training database: ", X_train.shape)
print("Size of the X validation database: ", X_val.shape)
print("Size of the X testing database: ", X_test.shape)
print("Size of the y training database: ", y_train.shape)
print("Size of the y validation database: ", y_val.shape)
print("Size of the y testing database: ", y_test.shape)

## <b>Step 6:</b> Training & evaluation

### <b>Fitting a base model</b>

In [ ]:
#- Start a new MLflow run
mlflow.set_experiment("baseline_model")

In [ ]:
with mlflow.start_run():
    #- Define the baseline model
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('regressor', LinearRegression())  # Use Linear Regression as the model
    ])

    # Fit the pipeline to training data
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_val)
    
    # Calculate evaluation metrics for regression
    mse = mean_squared_error(y_val, y_pred)
    mae = mean_absolute_error(y_val, y_pred)
    r2 = r2_score(y_val, y_pred)

    print(f"\nMean Squared Error (MSE): {mse:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"R² Score: {r2:.4f}")
    
    # Log metrics with MLflow
    mlflow.log_metric('mean_squared_error', mse)
    mlflow.log_metric('mean_absolute_error', mae)
    mlflow.log_metric('r2_score', r2)
    
    # Residuals plot
    residuals = y_val - y_pred
    plt.figure(figsize=(10, 6))
    sns.residplot(x=y_val, y=residuals, lowess=True, color="blue")
    plt.axhline(0, linestyle="--", color="red")
    plt.title("Residuals Plot")
    plt.xlabel("True Values")
    plt.ylabel("Residuals")
    plt.grid(True)
    plt.savefig("residuals_plot.png")
    plt.show()
    
    # Log Residuals Plot as artifact
    mlflow.log_artifact("residuals_plot.png")
    
    # Predicted vs True values plot
    plt.figure(figsize=(10, 6))
    plt.scatter(y_val, y_pred, alpha=0.7, color="green")
    plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], '--r', linewidth=2)
    plt.title("Predicted vs True Values")
    plt.xlabel("True Values")
    plt.ylabel("Predicted Values")
    plt.grid(True)
    plt.savefig("predicted_vs_true.png")
    plt.show()
    
    # Log Predicted vs True Plot as artifact
    mlflow.log_artifact("predicted_vs_true.png")
    
    # Save the model and log it with MLflow
    mlflow.sklearn.log_model(pipeline, "baseline_model")


### <b>Fitting an hyper-optimized model</b>

In [ ]:
#- Start a new MLflow run
mlflow.set_experiment("optimized_model")

In [ ]:
#- Define regressors and their parameter grids
regressors = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "ElasticNet": ElasticNet(),
    "Polynomial Regression": Pipeline([
        ("poly_features", PolynomialFeatures(include_bias=False)),
        ("regressor", LinearRegression())
    ]),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBoost Regressor": xgb.XGBRegressor(),
    "Support Vector Regressor": SVR(),
    "K-Nearest Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree Regressor": DecisionTreeRegressor(),
    "Gradient Boosting Regressor": GradientBoostingRegressor()
}

In [ ]:
def optuna_mlflow_tuner(trial):
    """
    Function to optimize hyperparameters of a wide range of regressors using Optuna 
    and log detailed results with MLflow. Combines advanced feature engineering, 
    standardized scaling, and regression evaluation with detailed metrics and visualizations.

    Parameters:
    -----------
    trial : optuna.trial.Trial
        The trial object provided by Optuna to suggest hyperparameters and track the results.

    Returns:
    --------
    float
        The main evaluation metric (Mean Squared Error) of the model on the validation set.
    """
    global best_model

    # Start an MLflow run for this trial
    with mlflow.start_run():
        # Select a regressor
        model_name = trial.suggest_categorical("model", list(regressors.keys()))
        regressor = regressors[model_name]

        # Suggest hyperparameters specific to the selected regressor
        if model_name == "Linear Regression":
            fit_intercept = trial.suggest_categorical("fit_intercept", [True, False])
            regressor.set_params(fit_intercept=fit_intercept)

        elif model_name == "Ridge Regression":
            alpha = trial.suggest_loguniform("ridge_alpha", 0.001, 100)
            regressor.set_params(alpha=alpha)

        elif model_name == "Lasso Regression":
            alpha = trial.suggest_loguniform("lasso_alpha", 0.001, 100)
            regressor.set_params(alpha=alpha)

        elif model_name == "ElasticNet":
            alpha = trial.suggest_loguniform("elasticnet_alpha", 0.001, 100)
            l1_ratio = trial.suggest_uniform("elasticnet_l1_ratio", 0.1, 1.0)
            regressor.set_params(alpha=alpha, l1_ratio=l1_ratio)

        elif model_name == "Polynomial Regression":
            degree = trial.suggest_int("polynomial_degree", 2, 5)
            regressor.set_params(poly_features__degree=degree)

        elif model_name == "Random Forest Regressor":
            n_estimators = trial.suggest_int("n_estimators", 50, 500, step=50)
            max_depth = trial.suggest_int("max_depth", 3, 20)
            min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
            min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)
            regressor.set_params(
                n_estimators=n_estimators,
                max_depth=max_depth,
                min_samples_split=min_samples_split,
                min_samples_leaf=min_samples_leaf,
                n_jobs=-1
            )

        elif model_name == "XGBoost Regressor":
            max_depth = trial.suggest_int("max_depth", 3, 15)
            learning_rate = trial.suggest_loguniform("learning_rate", 0.01, 0.5)
            n_estimators = trial.suggest_int("n_estimators", 50, 500, step=50)
            subsample = trial.suggest_uniform("subsample", 0.5, 1.0)
            colsample_bytree = trial.suggest_uniform("colsample_bytree", 0.5, 1.0)
            regressor.set_params(
                max_depth=max_depth,
                learning_rate=learning_rate,
                n_estimators=n_estimators,
                subsample=subsample,
                colsample_bytree=colsample_bytree
            )

        elif model_name == "Support Vector Regressor":
            kernel = trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"])
            C = trial.suggest_loguniform("C", 0.1, 100)
            epsilon = trial.suggest_loguniform("epsilon", 0.01, 1.0)
            regressor.set_params(kernel=kernel, C=C, epsilon=epsilon)

        elif model_name == "K-Nearest Neighbors Regressor":
            n_neighbors = trial.suggest_int("n_neighbors", 1, 50)
            weights = trial.suggest_categorical("weights", ["uniform", "distance"])
            algorithm = trial.suggest_categorical("algorithm", ["auto", "ball_tree", "kd_tree", "brute"])
            regressor.set_params(n_neighbors=n_neighbors, weights=weights, algorithm=algorithm)

        elif model_name == "Decision Tree Regressor":
            max_depth = trial.suggest_int("max_depth", 3, 20)
            min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
            min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)
            regressor.set_params(max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

        elif model_name == "Gradient Boosting Regressor":
            learning_rate = trial.suggest_loguniform("learning_rate", 0.01, 0.5)
            n_estimators = trial.suggest_int("n_estimators", 50, 500, step=50)
            max_depth = trial.suggest_int("max_depth", 3, 15)
            regressor.set_params(learning_rate=learning_rate, n_estimators=n_estimators, max_depth=max_depth)

        # Build the pipeline
        pipeline = Pipeline([
            ("feature_engineering", FunctionTransformer(do_feature_eng, validate=False)),  # Replace with your feature engineering function
            ("scaler", StandardScaler()),
            ("regressor", regressor)
        ])

        # Train the pipeline
        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_val)

        # Calculate evaluation metrics for regression
        mse = mean_squared_error(y_val, y_pred)
        mae = mean_absolute_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)

        # Log metrics in MLflow
        mlflow.log_metric("mean_squared_error", mse)
        mlflow.log_metric("mean_absolute_error", mae)
        mlflow.log_metric("r2_score", r2)
        mlflow.log_param("model", model_name)

        # Log hyperparameters
        params = regressor.get_params()
        for param_name, param_value in params.items():
            mlflow.log_param(param_name, param_value)

        # Plot predicted vs actual values
        plt.figure(figsize=(10, 6))
        plt.scatter(y_val, y_pred, alpha=0.6, color="blue")
        plt.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], '--r', linewidth=2)
        plt.title(f"Predicted vs Actual Values - {model_name}")
        plt.xlabel("Actual Values")
        plt.ylabel("Predicted Values")
        plt.grid(True)
        plt.savefig("predicted_vs_actual.png")
        mlflow.log_artifact("predicted_vs_actual.png")
        plt.close()

        # Save the model if it's the best one found so far
        if trial.number == 0 or mse < study.best_value:
            best_model = pipeline

        return mse


In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(optuna_mlflow_tuner, n_trials=50)

print("\nBest hyperparameters found:", study.best_params)

### <b>Testing</b>

In [ ]:
best_model.fit(pre_X_train, pre_y_train)

In [ ]:
def evaluate_model_on_test(model, X_test, y_test):
    """
    Evaluate a trained regression model on the test set and log metrics and visualizations.
    
    Parameters:
    -----------
    model : sklearn model
        The trained machine learning model (pipeline or standalone model).
    X_test : array-like
        The test set features.
    y_test : array-like
        The true labels of the test set.
    
    Returns:
    --------
    dict
        A dictionary containing evaluation metrics (MSE, MAE, R² Score).
    """
    # Generate predictions
    y_pred = model.predict(X_test)

    # Calculate performance metrics
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Print metrics
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"R² Score: {r2:.4f}")

    # Residuals plot
    residuals = y_test - y_pred
    plt.figure(figsize=(10, 6))
    sns.residplot(x=y_test, y=residuals, lowess=True, color="blue")
    plt.axhline(0, linestyle="--", color="red")
    plt.title("Residuals Plot")
    plt.xlabel("True Values")
    plt.ylabel("Residuals")
    plt.grid(True)
    plt.show()

    # Predicted vs True Values plot
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.7, color="green")
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', linewidth=2)
    plt.title("Predicted vs True Values")
    plt.xlabel("True Values")
    plt.ylabel("Predicted Values")
    plt.grid(True)
    plt.show()

    # Optional: Return the metrics as a dictionary
    return {
        "mean_squared_error": mse,
        "mean_absolute_error": mae,
        "r2_score": r2
    }


In [ ]:
evaluate_model_on_test(best_model, X_test, y_test)

In [ ]:
def cross_validate_model(model, X, y, cv=5, scoring="neg_mean_squared_error"):
    """
    Perform cross-validation on a regression model and report averaged performance metrics.
    
    Parameters:
    -----------
    model : sklearn model
        The trained machine learning model (pipeline or standalone model).
    X : array-like
        The features of the dataset.
    y : array-like
        The target values of the dataset.
    cv : int, optional
        Number of folds for cross-validation. Default is 5.
    scoring : str, optional
        The scoring metric to use for cross-validation. Default is "neg_mean_squared_error".
        Common options: "r2", "neg_mean_squared_error", "neg_mean_absolute_error".
    
    Returns:
    --------
    dict
        A dictionary with cross-validated metrics.
    """
    # Define cross-validation strategy
    kfold = KFold(n_splits=cv, shuffle=True, random_state=42)

    # Perform cross-validation
    print(f"Performing {cv}-fold cross-validation...")
    scores = cross_val_score(model, X, y, cv=kfold, scoring=scoring)

    # Process and return results
    if scoring == "neg_mean_squared_error":
        scores = -scores  # Convert to positive MSE values
        metric_name = "Mean Squared Error (MSE)"
    elif scoring == "neg_mean_absolute_error":
        scores = -scores  # Convert to positive MAE values
        metric_name = "Mean Absolute Error (MAE)"
    elif scoring == "r2":
        metric_name = "R² Score"
    else:
        raise ValueError("Unsupported scoring metric. Use 'r2', 'neg_mean_squared_error', or 'neg_mean_absolute_error'.")

    avg_score = scores.mean()
    std_score = scores.std()

    print(f"{metric_name}: {avg_score:.4f} ± {std_score:.4f}")

    return {
        "metric_name": metric_name,
        "mean": avg_score,
        "std": std_score
    }

In [ ]:
# Example usage with full training data
cross_val_results = cross_validate_model(best_model, X_train, y_train, cv=5, scoring="neg_mean_squared_error")

## <b>Step 7:</b> Prediction

In [ ]:
#- Retrain the model on the whole dataset 
best_model.fit(X, y)

In [ ]:
X_to_predict = [[]]
predictions = best_model.predict(X_to_predict)
print("Predictions:", predictions)

In [ ]:
data_path_2 = ".csv"
data_2 = pd.read_csv(data_path_2)
raw_data_2 = data_2.copy()
data_2.head(3)

In [ ]:
basic_inspection(data_2, visualize_nulls=True, detailed_summary=True, check_duplicates=True)

In [ ]:
X_features = ["feature1", "feature2", "feature3"]  # Replace with actual feature names
X_to_predict = data_2[X_features]
data_2 = data_cleaning(X_to_predict)

In [ ]:
predictions = best_model.predict(X_to_predict)
print("Predictions:")
print(predictions)

In [ ]:
# Add predictions to the original data
results_df = raw_data_2.copy()  # Copy the original data for context
results_df["Predicted Values"] = predictions  # Add predicted values as a new column

results_file_path = "predictions_results.csv"
results_df.to_csv(results_file_path, index=False)

print(f"Results exported successfully to {results_file_path}")